In [52]:
import numpy as np
import cv2
import heapq

In [53]:
h = 50*4 #50 mm scaled up
w = 180*4 #50 mm scaled up



map = np.zeros((h, w, 3), dtype=np.uint8)

clearance = 5 #mm
radius = 5 #mm
buffer = (clearance + radius)*4

map_limits = map.copy()
map_limits = cv2.cvtColor(map_limits, cv2.COLOR_BGR2GRAY)
map_limits = cv2.rectangle(map_limits, (0,0), (w,h), (255), 1)
locations = np.where(map_limits == (255))

boundary = set()
for i in range(0,len(locations[0])):
    for a in [0, 30, 2*30, 3*30, 4*30, 5*30, 6*30, 7*30, 8*30, 9*30, 10*30, 11*30]:
        boundary.add((locations[0][i], locations[1][i], a))


del map_limits
del locations


In [54]:
def add_boundary(inpt_x,inpt_y):
    # add given point to the boundary sets
    boundary.add((inpt_x,inpt_y))

    # add all points in a box around given point to boundary sets
    for x in range(inpt_x-buffer, inpt_x+(buffer+1)):
        for y in range(inpt_y-buffer, inpt_y+(buffer+1)):
            for a in [0, 30, 2*30, 3*30, 4*30, 5*30, 6*30, 7*30, 8*30, 9*30, 10*30, 11*30]:
                boundary.add((x,y,a))

In [55]:
# define obstacles
for y in range(17*4,35*4):
    # define obstacle E
    for x in range(20*4,40*4):
        if 16*4 < y <= 20*4:
            if 21*4 < x <= 39*4:
                # top horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 20*4 < y <= 23*4:
            if 21*4 < x <= 25*4:
                # between top and middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 23*4 < y <=27*4:
            if 21*4 < x <= 39*4:
                # middle horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 27*4 < y <= 30*4:
            if 21*4 < x <= 25*4:
                # between middle and bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        if 30*4 < y <= 34*4:
            if 21*4 < x <= 39*4:
                #bottom horizontal of E
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle N
    for x in range(40*4,60*4):
        if 41*4 < x <= 45*4:
            # first vertical of N
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # second vertical of N
        if 55*4 < x <= 59*4:
            map[y,x] = (255,0,0)
            add_boundary(x,y)

            # diagonal of N determined by equations of two lines
        if 45*4 < x <= 55*4:
            if ((1.2*x)-38*4)< y <= ((1.2*x)-32*4):
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle P
    for x in range(60*4, 80*4):

        # main vertical of P
        if 61*4 < x <=67*4:
            map[y,x] = (255,0,0)
            add_boundary(x,y)

        # round part of P determined by ellipse equation
        if 67*4 < x <= 79*4:
            if (((x-67*4)**2)/((12*4)**2))+(((y-22*4)**2)/((6*4)**2)) <= 1:
                map[y,x] = (255,0,0)
                add_boundary(x,y)
    
    # define obstacle M
    for x in range(80*4,100*4):

        # first vertical of M
        if 81*4 < x <=85*4:
            map[y,x] = (255,0,0)
            add_boundary(x,y)
        
        # first angle of M determined by equations of two lines
        if 85*4 < x <= 90*4:
            if (((9/5)*x)-137*4) < y <= (((9/5)*x)-129*4):
                map[y,x] = (255,0,0)
                add_boundary(x,y)

        # second angle of M determined by equations of two lines
        if 90*4 < x <= 95*4:
            if (((-9/5)*x)+187*4) < y <= (((-9/5)*x)+195*4):
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        
        # last vertical of M 
        if 95*4< x <= 99*4:
            map[y,x] = (255,0,0)
            add_boundary(x,y)
    
    # define obstacle 6
    for x in range(100*4,120*4):

        # draw main circle per equation
        if ((x-110*4)**2)+(y-28.5*4)**2 <= (5.5*4)**2:
            # add both 6's spaced 20 pixels apart
            map[y,x] = (255,0,0)
            map[y,x+20*4] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+20*4,y)

        # draw second circle at tip of "tail" of 6
        if ((x-110.85*4)**2)+(y-18*4)**2 <= (2*4)**2:
            map[y,x] = (255,0,0)
            map[y,x+20*4] = (255,0,0)
            add_boundary(x,y)
            add_boundary(x+20*4,y)
        
        # draw part of a ring defined by two circles and two lines
        # to define the "tail" of 6
        if x-92.85*4 < y <=28.5*4:
            if (12.85*4)**2 < (((x-121.35*4)**2)+((y-28.5*4)**2)) <= (16.85*4)**2:
                map[y,x] = (255,0,0)
                map[y,x+20*4] = (255,0,0)
                add_boundary(x,y)
                add_boundary(x+20*4,y)

    # define obstacle 1
    for x in range(140*4,160*4):
        if 16*4 < y <= 20.62*4:
            if 144*4 < x <= 152*4:
                # top horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if 20.62*4 < y <= 30*4:
            if 148*4 < x <= 152*4:
                # main vertical of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)
        if 30*4 < y <= 34*4:
            if 144*4 < x < 156*4:
                # bottom horizontal of 1
                map[y,x] = (255,0,0)
                add_boundary(x,y)

KeyboardInterrupt: 

In [ ]:
cv2.imshow("map",map)
cv2.waitKey(0)
cv2.destroyAllWindows()